In [1]:
import numpy as np
from copy import deepcopy
import random
import time
import os, psutil

# Heuristic Functions
There are two heuristics:
- h1 - Heuristic using Manhattan distance
- h2 - Heuristic using number of misplaced tiles

0 represents a blank tile.

In [2]:
def h1(curr_state, goal_state):
    
    goal_indices = np.argsort(goal_state.reshape(-1,1), axis=0)
    curr_indices = np.argsort(curr_state.reshape(-1,1), axis=0)
    x = (abs(goal_indices // 3 - curr_indices // 3))
    y = (abs(goal_indices % 3 - curr_indices % 3))
    h = np.sum(x + y)
    return h

In [3]:
def h2(curr_state, goal_state):
    
    h = np.sum(curr_state != goal_state)
    return h

# Generating random instance of the puzzle at depth d form the goal state

In [4]:
def generate_instance(goal_state, depth, debug=False):
   
    
    x_prev, y_prev = np.array(np.where(goal_state == 0)).reshape(-1)
    curr_state = np.copy(goal_state)
    
    visited = np.array([curr_state.reshape(-1)])
    n = depth
    if debug == True:
        print(f"Depth: {depth-n}")
        print(f"h1 value: {h1(curr_state, goal_state)}\th2 value : {h2(curr_state, goal_state)}")
        print(curr_state, end='\n\n')
    while(n):
        
        possible_states = []
        if x_prev > 0:
            possible_states.append([x_prev-1, y_prev])
        if x_prev < 2:
            possible_states.append([x_prev+1, y_prev])
        if y_prev > 0:
            possible_states.append([x_prev, y_prev-1])
        if y_prev < 2:
            possible_states.append([x_prev, y_prev+1])
        
        x_new, y_new = random.choice(possible_states)
        
        curr_state[x_new, y_new], curr_state[x_prev, y_prev] = curr_state[x_prev, y_prev], curr_state[x_new, y_new]
        if curr_state.reshape(-1).tolist() not in visited.tolist():
            
            visited = np.vstack((visited, curr_state.reshape(-1)))
            n -= 1
            if debug == True:
                if x_new > x_prev:
                    step = "Down"
                elif x_new < x_prev:
                    step = "Up"
                elif y_new > y_prev:
                    step = "Right"
                else:
                    step = "Left"
                print(f"Depth: {depth-n}\tStep taken to reach: {step}")
                print(f"h1 value: {h1(curr_state, goal_state)}\th2 value : {h2(curr_state, goal_state)}")
                print(curr_state, end='\n\n')
            x_prev, y_prev = x_new, y_new
        else:
            
            curr_state[x_new, y_new], curr_state[x_prev, y_prev] = curr_state[x_prev, y_prev], curr_state[x_new, y_new]
    return curr_state


# Verifying Heuristics and Instance genration

In [5]:
GOAL_STATE = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
print("Goal State:\n", GOAL_STATE)

Goal State:
 [[1 2 3]
 [8 0 4]
 [7 6 5]]


In [6]:
# Generating random instance
depth = 16
CURR_STATE = generate_instance(GOAL_STATE, depth, debug=False)
print(CURR_STATE)

[[6 1 0]
 [7 3 2]
 [4 8 5]]


In [7]:
# Heuristic 1
print("Value from h1:", h1(CURR_STATE, GOAL_STATE))

Value from h1: 16


In [8]:
# Heuristic 2
print("Value from h2:", h2(CURR_STATE, GOAL_STATE))

Value from h2: 8


# General Experiment

In [9]:
 def find_pos(matrix,value):
     if value < 0 or value > 8:
       raise Exception ("Give the value is out of range")
     else:
       # print(matrix, value)
       return np.array(np.where(matrix == value )).reshape(-1)


In [10]:
 def swap_tiles(matrix,next_pos):
     row,col=find_pos(matrix,0)

     matrix[row, col], matrix[next_pos[0], next_pos[1]] = matrix[next_pos[0], next_pos[1]], matrix[row, col]

     return matrix


In [11]:
 def possible_moves(current_state):
     row,col=find_pos(current_state,0)
     ans_list=[]
     if(row>0):
         ans_list.append([row-1,col])
     if(row<2):
         ans_list.append([row+1,col])
     if(col>0):
         ans_list.append([row,col-1])
     if(col<2):
         ans_list.append([row,col+1])
     return ans_list

In [12]:
  visited = np.array([0]*9)

In [13]:
 def solve(curr_state,visited):
     next_possible=[]
     heurestic_val=[]
     choosen_matrix=curr_state
     visited = np.vstack((visited, choosen_matrix.reshape(-1)))
     if (choosen_matrix == GOAL_STATE).all():
         print("REACHED THE GOAL STATE")
         return 
     for i in possible_moves(curr_state):
        if np.array(i).reshape(-1).tolist() not in visited.tolist():
         matrix2=deepcopy(curr_state)
         temp=swap_tiles(matrix2,i)
         next_possible.append(temp)

         c=h1(temp,GOAL_STATE)
         heurestic_val.append(c)
     for k in range(len(next_possible)):
         if(heurestic_val[k]==min(heurestic_val)):
             visited = np.vstack((visited, next_possible[k].reshape(-1)))
             choosen_matrix=next_possible[k]
     print(choosen_matrix)
     if (choosen_matrix == GOAL_STATE).all():
         print("REACHED THE GOAL STATE")
         return
     else:
         print(visited)
         solve(choosen_matrix,visited)



In [14]:
CURR_STATE = generate_instance(GOAL_STATE, 10)

In [15]:
print(CURR_STATE)

[[8 1 2]
 [7 6 3]
 [5 4 0]]


In [16]:
 solve(CURR_STATE, visited)
 print(visited)

Streaming output truncated to the last 5000 lines.
[[8 1 2]
 [7 3 0]
 [5 6 4]]
[[0 0 0 ... 0 0 0]
 [8 1 2 ... 5 4 0]
 [8 1 2 ... 5 0 4]
 ...
 [8 1 2 ... 5 0 4]
 [8 1 2 ... 5 6 4]
 [8 1 2 ... 5 6 4]]
[[8 1 2]
 [7 0 3]
 [5 6 4]]
[[0 0 0 ... 0 0 0]
 [8 1 2 ... 5 4 0]
 [8 1 2 ... 5 0 4]
 ...
 [8 1 2 ... 5 6 4]
 [8 1 2 ... 5 6 4]
 [8 1 2 ... 5 6 4]]
[[8 1 2]
 [7 3 0]
 [5 6 4]]
[[0 0 0 ... 0 0 0]
 [8 1 2 ... 5 4 0]
 [8 1 2 ... 5 0 4]
 ...
 [8 1 2 ... 5 0 4]
 [8 1 2 ... 5 6 4]
 [8 1 2 ... 5 6 4]]
[[8 1 2]
 [7 0 3]
 [5 6 4]]
[[0 0 0 ... 0 0 0]
 [8 1 2 ... 5 4 0]
 [8 1 2 ... 5 0 4]
 ...
 [8 1 2 ... 5 6 4]
 [8 1 2 ... 5 6 4]
 [8 1 2 ... 5 6 4]]
[[8 1 2]
 [7 3 0]
 [5 6 4]]
[[0 0 0 ... 0 0 0]
 [8 1 2 ... 5 4 0]
 [8 1 2 ... 5 0 4]
 ...
 [8 1 2 ... 5 0 4]
 [8 1 2 ... 5 6 4]
 [8 1 2 ... 5 6 4]]
[[8 1 2]
 [7 0 3]
 [5 6 4]]
[[0 0 0 ... 0 0 0]
 [8 1 2 ... 5 4 0]
 [8 1 2 ... 5 0 4]
 ...
 [8 1 2 ... 5 6 4]
 [8 1 2 ... 5 6 4]
 [8 1 2 ... 5 6 4]]
[[8 1 2]
 [7 3 0]
 [5 6 4]]
[[0 0 0 ... 0 0 0]
 [8 1 2 ... 5 

RecursionError: ignored

In [17]:
 def get_possible_moves(curr_state):
 
     row, col = np.array(np.where(curr_state == 0)).reshape(-1)
     possible_moves = []
     if(row > 0):
         next_state = curr_state.copy()
         next_state[row, col], next_state[row-1, col] = next_state[row-1, col], next_state[row, col]
         possible_moves.append(next_state)
     if(row < 2):
         next_state = curr_state.copy()
         next_state[row, col], next_state[row+1, col] = next_state[row+1, col], next_state[row, col]
         possible_moves.append(next_state)
     if(col > 0):
         next_state = curr_state.copy()
         next_state[row, col], next_state[row, col-1] = next_state[row, col-1], next_state[row, col]
         possible_moves.append(next_state)
     if(col < 2):
         next_state = curr_state.copy()
         next_state[row, col], next_state[row, col+1] = next_state[row, col+1], next_state[row, col]
         possible_moves.append(next_state)
     return possible_moves

In [18]:
 def sort_by_heuristic(possible_moves, goal_state, heuristic):

     if heuristic == 1:
         sorted_possible_moves = sorted(possible_moves, key=lambda x: h1(x, goal_state))
     if heuristic == 2:
         sorted_possible_moves = sorted(possible_moves, key=lambda x:h2(x, goal_state))
     return sorted_possible_moves

In [19]:
 def solve2(curr_state, goal_state, heuristic=0):
     visited = np.array([])
     frontier = np.array([curr_state.reshape(-1)])
     while (curr_state.tolist() != goal_state.tolist()):
         print(curr_state, end="\n\n")
         possible_moves = get_possible_moves(curr_state)
         if heuristic != 0:
             possible_moves = sort_by_heuristic(possible_moves, goal_state, heuristic)
         for move in possible_moves:
             curr_move = move.reshape(-1).tolist()
             if curr_move not in frontier.tolist():
                 if curr_move not in visited.tolist():
                     frontier = np.vstack((frontier, move.reshape(-1)))
         if (frontier == np.array([])):
             print("STRUCK!!!")
             return
         if (visited == np.array([])).all():
             visited = np.array(curr_state.reshape(-1,1))
         else:
             visited = np.vstack((visited, curr_state.reshape(-1)))
         frontier = np.delete(frontier, 0, 0)
         curr_state = frontier[0].reshape(3,3)
     print("GOAL REACHED!!!")
     print(curr_state)

In [20]:
CURR_STATE = generate_instance(GOAL_STATE, 8)
print(CURR_STATE)

[[8 3 0]
 [7 1 4]
 [6 2 5]]


In [21]:
solve2(CURR_STATE, GOAL_STATE, 1)

[[8 3 0]
 [7 1 4]
 [6 2 5]]

[[8 0 3]
 [7 1 4]
 [6 2 5]]

[[8 3 4]
 [7 1 0]
 [6 2 5]]

[[8 1 3]
 [7 0 4]
 [6 2 5]]

[[0 8 3]
 [7 1 4]
 [6 2 5]]

[[8 3 0]
 [7 1 4]
 [6 2 5]]

[[8 3 4]
 [7 0 1]
 [6 2 5]]

[[8 3 4]
 [7 1 5]
 [6 2 0]]

[[8 1 3]
 [7 2 4]
 [6 0 5]]

[[8 0 3]
 [7 1 4]
 [6 2 5]]

[[8 1 3]
 [0 7 4]
 [6 2 5]]

[[8 1 3]
 [7 4 0]
 [6 2 5]]

[[7 8 3]
 [0 1 4]
 [6 2 5]]

[[8 3 4]
 [7 1 0]
 [6 2 5]]

[[8 3 4]
 [7 2 1]
 [6 0 5]]

[[8 0 4]
 [7 3 1]
 [6 2 5]]

[[8 3 4]
 [0 7 1]
 [6 2 5]]

[[8 3 4]
 [7 1 5]
 [6 0 2]]

[[8 1 3]
 [7 0 4]
 [6 2 5]]

[[8 1 3]
 [7 2 4]
 [0 6 5]]

[[8 1 3]
 [7 2 4]
 [6 5 0]]

[[0 8 3]
 [7 1 4]
 [6 2 5]]

[[8 3 0]
 [7 1 4]
 [6 2 5]]

[[0 1 3]
 [8 7 4]
 [6 2 5]]

[[8 1 3]
 [6 7 4]
 [0 2 5]]

[[8 1 0]
 [7 4 3]
 [6 2 5]]

[[8 1 3]
 [7 4 5]
 [6 2 0]]

[[7 8 3]
 [1 0 4]
 [6 2 5]]

[[7 8 3]
 [6 1 4]
 [0 2 5]]

[[8 3 4]
 [7 0 1]
 [6 2 5]]

[[8 3 4]
 [7 1 5]
 [6 2 0]]

[[8 3 4]
 [7 2 1]
 [0 6 5]]

[[8 3 4]
 [7 2 1]
 [6 5 0]]

[[0 8 4]
 [7 3 1]
 [6 2 5]]

[[8 4 0]
 [7 3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


[[8 3 4]
 [0 1 5]
 [7 6 2]]

[[3 0 4]
 [8 2 1]
 [7 6 5]]

[[8 3 4]
 [2 1 0]
 [7 6 5]]

[[8 0 4]
 [2 3 1]
 [7 6 5]]

[[8 3 4]
 [2 6 1]
 [7 0 5]]

[[8 0 3]
 [7 2 4]
 [6 5 1]]

[[8 0 4]
 [7 3 2]
 [6 5 1]]

[[8 3 4]
 [7 5 2]
 [6 0 1]]

[[8 3 4]
 [0 7 2]
 [6 5 1]]

[[7 0 4]
 [3 8 1]
 [6 2 5]]

[[7 8 4]
 [3 2 1]
 [6 0 5]]

[[7 8 4]
 [3 1 0]
 [6 2 5]]

[[7 8 4]
 [6 3 1]
 [2 0 5]]

[[8 4 1]
 [7 3 5]
 [6 0 2]]

[[3 7 4]
 [8 2 1]
 [6 0 5]]

[[3 7 4]
 [8 1 0]
 [6 2 5]]

[[3 7 4]
 [0 8 1]
 [6 2 5]]

[[3 4 1]
 [8 7 0]
 [6 2 5]]

[[8 3 4]
 [0 6 1]
 [2 7 5]]

[[8 3 4]
 [6 1 0]
 [2 7 5]]

[[8 0 4]
 [6 3 1]
 [2 7 5]]

[[8 3 4]
 [6 7 0]
 [2 5 1]]

[[7 8 4]
 [0 3 5]
 [6 1 2]]

[[8 4 5]
 [7 3 0]
 [6 1 2]]

[[3 0 4]
 [8 7 5]
 [6 1 2]]

[[8 3 4]
 [6 7 5]
 [1 0 2]]

[[8 0 3]
 [7 5 4]
 [6 1 2]]

[[8 0 4]
 [1 3 5]
 [7 6 2]]

[[8 3 4]
 [1 6 5]
 [7 0 2]]

[[8 3 4]
 [1 5 0]
 [7 6 2]]

[[3 0 4]
 [8 1 5]
 [7 6 2]]

[[8 1 3]
 [7 0 4]
 [6 2 5]]

[[8 1 3]
 [7 2 4]
 [0 6 5]]

[[8 1 3]
 [7 2 4]
 [6 5 0]]

[[0 8 3]
 [7 1

Works for inputs upto atleast 16

# Task#1: Generate Puzzle-8 instances with the goal state at depth “d”

In [22]:
CURR_STATE32 = generate_instance(GOAL_STATE, 32)
print(CURR_STATE32)

[[2 3 7]
 [6 8 4]
 [0 1 5]]


In [23]:

solve2(CURR_STATE32, GOAL_STATE, 1)

[[1 2 7]
 [6 8 4]
 [3 0 5]]

[[1 0 2]
 [8 4 7]
 [6 3 5]]

[[1 2 7]
 [8 4 5]
 [6 0 3]]

[[1 0 7]
 [8 2 3]
 [6 5 4]]

[[1 2 7]
 [8 5 3]
 [6 0 4]]

[[1 2 7]
 [0 8 3]
 [6 5 4]]

[[1 0 2]
 [8 3 7]
 [6 5 4]]

[[1 0 4]
 [3 7 2]
 [8 6 5]]

[[1 7 4]
 [0 3 2]
 [8 6 5]]

[[1 7 4]
 [3 6 2]
 [8 0 5]]

[[1 7 4]
 [3 2 5]
 [8 0 6]]

[[1 0 7]
 [6 2 4]
 [3 8 5]]

[[1 2 7]
 [6 4 0]
 [3 8 5]]

[[2 0 7]
 [1 6 4]
 [3 8 5]]

[[1 0 2]
 [3 6 7]
 [8 5 4]]

[[1 0 7]
 [3 2 6]
 [8 5 4]]

[[1 2 7]
 [3 5 6]
 [8 0 4]]

[[1 4 2]
 [0 3 7]
 [8 6 5]]

[[1 4 2]
 [3 7 0]
 [8 6 5]]

[[1 4 2]
 [3 6 7]
 [8 0 5]]

[[3 1 2]
 [0 4 7]
 [8 6 5]]

[[1 0 7]
 [3 2 5]
 [8 4 6]]

[[1 2 7]
 [3 5 0]
 [8 4 6]]

[[1 2 7]
 [0 4 5]
 [3 8 6]]

[[1 0 4]
 [7 2 3]
 [8 6 5]]

[[1 2 4]
 [7 6 3]
 [8 0 5]]

[[1 2 4]
 [7 3 0]
 [8 6 5]]

[[1 2 4]
 [8 7 3]
 [6 0 5]]

[[2 0 7]
 [1 6 4]
 [8 5 3]]

[[2 0 4]
 [1 7 6]
 [8 5 3]]

[[2 7 4]
 [1 5 6]
 [8 0 3]]

[[2 7 4]
 [0 1 6]
 [8 5 3]]

[[1 2 4]
 [0 7 5]
 [8 3 6]]

[[2 4 5]
 [1 7 0]
 [8 3 6]]

[[2 7 4]
 [8 1

KeyboardInterrupt: ignored

In [24]:
CURR_STATE64 = generate_instance(GOAL_STATE, 64)
print(CURR_STATE64)

[[8 2 4]
 [6 0 1]
 [5 7 3]]


In [25]:

solve2(CURR_STATE64, GOAL_STATE, 1)

[[8 2 4]
 [6 0 1]
 [5 7 3]]

[[8 2 4]
 [0 6 1]
 [5 7 3]]

[[8 2 4]
 [6 1 0]
 [5 7 3]]

[[8 0 4]
 [6 2 1]
 [5 7 3]]

[[8 2 4]
 [6 7 1]
 [5 0 3]]

[[0 2 4]
 [8 6 1]
 [5 7 3]]

[[8 2 4]
 [6 0 1]
 [5 7 3]]

[[8 2 4]
 [5 6 1]
 [0 7 3]]

[[8 2 0]
 [6 1 4]
 [5 7 3]]

[[8 2 4]
 [6 1 3]
 [5 7 0]]

[[0 8 4]
 [6 2 1]
 [5 7 3]]

[[8 4 0]
 [6 2 1]
 [5 7 3]]

[[8 2 4]
 [6 7 1]
 [0 5 3]]

[[8 2 4]
 [6 7 1]
 [5 3 0]]

[[8 2 4]
 [0 6 1]
 [5 7 3]]

[[2 0 4]
 [8 6 1]
 [5 7 3]]

[[8 2 4]
 [6 1 0]
 [5 7 3]]

[[8 0 4]
 [6 2 1]
 [5 7 3]]

[[8 2 4]
 [6 7 1]
 [5 0 3]]

[[8 2 4]
 [5 6 1]
 [7 0 3]]

[[8 0 2]
 [6 1 4]
 [5 7 3]]

[[8 2 4]
 [6 1 3]
 [5 0 7]]

[[6 8 4]
 [0 2 1]
 [5 7 3]]

[[8 4 1]
 [6 2 0]
 [5 7 3]]

[[8 2 4]
 [0 7 1]
 [6 5 3]]

[[8 2 4]
 [6 7 0]
 [5 3 1]]

[[0 2 4]
 [8 6 1]
 [5 7 3]]

[[8 2 4]
 [6 0 1]
 [5 7 3]]

[[8 2 4]
 [5 6 1]
 [0 7 3]]

[[2 6 4]
 [8 0 1]
 [5 7 3]]

[[2 4 0]
 [8 6 1]
 [5 7 3]]

[[8 2 0]
 [6 1 4]
 [5 7 3]]

[[8 2 4]
 [6 1 3]
 [5 7 0]]

[[0 8 4]
 [6 2 1]
 [5 7 3]]

[[8 4 0]
 [6 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


Streaming output truncated to the last 5000 lines.
[[0 1 2]
 [5 7 4]
 [8 6 3]]

[[1 7 2]
 [5 6 4]
 [0 8 3]]

[[1 7 2]
 [5 6 4]
 [8 3 0]]

[[1 7 2]
 [5 4 3]
 [8 6 0]]

[[1 7 0]
 [5 4 2]
 [8 6 3]]

[[7 5 2]
 [1 0 4]
 [8 6 3]]

[[7 2 0]
 [1 5 4]
 [8 6 3]]

[[1 7 2]
 [8 3 5]
 [0 6 4]]

[[1 7 2]
 [8 3 5]
 [6 4 0]]

[[0 7 2]
 [1 8 5]
 [6 3 4]]

[[1 7 2]
 [6 8 5]
 [0 3 4]]

[[1 5 7]
 [8 0 2]
 [6 3 4]]

[[0 1 7]
 [8 5 2]
 [6 3 4]]

[[1 2 0]
 [6 7 4]
 [5 8 3]]

[[0 1 2]
 [6 7 4]
 [5 8 3]]

[[0 7 2]
 [1 6 4]
 [5 8 3]]

[[1 7 2]
 [6 4 3]
 [5 8 0]]

[[1 7 0]
 [6 4 2]
 [5 8 3]]

[[1 7 2]
 [6 0 8]
 [5 3 4]]

[[1 7 0]
 [6 8 2]
 [5 3 4]]

[[0 7 2]
 [1 8 3]
 [6 4 5]]

[[1 7 2]
 [6 8 3]
 [0 4 5]]

[[1 7 0]
 [8 3 2]
 [6 4 5]]

[[1 7 2]
 [4 0 3]
 [8 6 5]]

[[0 7 2]
 [1 4 3]
 [8 6 5]]

[[1 4 7]
 [8 2 3]
 [6 5 0]]

[[1 4 7]
 [8 5 2]
 [0 6 3]]

[[1 4 7]
 [8 5 2]
 [6 3 0]]

[[0 4 7]
 [1 8 2]
 [6 5 3]]

[[1 4 7]
 [6 8 2]
 [0 5 3]]

[[8 1 7]
 [4 0 2]
 [6 5 3]]

[[0 1 4]
 [8 2 3]
 [6 7 5]]

[[1 4 0]
 [8 2 3]
 [6

KeyboardInterrupt: ignored

# Task#2:Prepare a table indicating the memory and time requirements to solve Puzzle-8 instances (depth “d”) using your graph search agent.


### Time and memory requirement

In [27]:
import time
import os, psutil
from resource import *
import sys
import resource

In [28]:

sys.setrecursionlimit(10**6) 

print(f"pagesize: {resource.getpagesize()}")
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

pagesize: 4096


224276

In [30]:
depths = np.array([2,4,8])
print(depths)
for d in depths:
  start = time.perf_counter()
  CURR_STATE = generate_instance(GOAL_STATE, d)
  solve(CURR_STATE, visited)
  end = time.perf_counter()
  print(f"Depth: {d}\tTime Taken: {end-start}\tMax Mem: {resource.getrusage(resource.RUSAGE_SELF).ru_maxrss}")


Streaming output truncated to the last 5000 lines.
[[1 2 3]
 [8 6 0]
 [7 5 4]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]
 ...
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]]
[[1 2 3]
 [8 0 6]
 [7 5 4]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]
 ...
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 0]
 [1 2 3 ... 7 5 4]]
[[1 2 3]
 [8 6 0]
 [7 5 4]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]
 ...
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]]
[[1 2 3]
 [8 0 6]
 [7 5 4]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]
 ...
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 0]
 [1 2 3 ... 7 5 4]]
[[1 2 3]
 [8 6 0]
 [7 5 4]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]
 ...
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]]
[[1 2 3]
 [8 0 6]
 [7 5 4]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 4]
 ...
 [1 2 3 ... 7 5 4]
 [1 2 3 ... 7 5 0]
 [1 2 3 ... 7 5 4]]
[[1 2 3]
 [8 6 0]
 [7 5 4]]
[[0 0 0 ... 0 0 0]
 [1 2 3 ... 7 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-30-c72fa6c54363>", line 6, in <module>
    solve(CURR_STATE, visited)
  File "<ipython-input-13-dd821c6ef116>", line 27, in solve
    solve(choosen_matrix,visited)
  File "<ipython-input-13-dd821c6ef116>", line 27, in solve
    solve(choosen_matrix,visited)
  File "<ipython-input-13-dd821c6ef116>", line 27, in solve
    solve(choosen_matrix,visited)
  [Previous line repeated 3395 more times]
  File "<ipython-input-13-dd821c6ef116>", line 11, in solve
    matrix2=deepcopy(curr_state)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'Keyboard

KeyboardInterrupt: ignored

# Solving the Puzzle

In [31]:
def get_possible_moves(curr_state,parent):
   
    row, col = np.array(np.where(curr_state == 0)).reshape(-1)
    possible_moves = []
    possible_paths = []

    if(row > 0):
        next_state = curr_state.copy()
        next_state[row, col], next_state[row-1, col] = next_state[row-1, col], next_state[row, col]
        possible_moves.append(next_state)
        possible_paths.append(parent+"U->")
    if(row < 2):
        next_state = curr_state.copy()
        next_state[row, col], next_state[row+1, col] = next_state[row+1, col], next_state[row, col]
        possible_moves.append(next_state)
        possible_paths.append(parent+"D->")
    if(col > 0):
        next_state = curr_state.copy()
        next_state[row, col], next_state[row, col-1] = next_state[row, col-1], next_state[row, col]
        possible_moves.append(next_state)
        possible_paths.append(parent+"L->")
    if(col < 2):
        next_state = curr_state.copy()
        next_state[row, col], next_state[row, col+1] = next_state[row, col+1], next_state[row, col]
        possible_moves.append(next_state)
        possible_paths.append(parent+"R->")
    return possible_moves,possible_paths

In [32]:
def sort_by_heuristic(possible_moves,possible_paths, goal_state, heuristic):
    
    if heuristic == 1:
         sorted_possible_moves,sorted_possible_paths = zip(*sorted(zip(possible_moves,possible_paths),key=lambda x: h1(x[0], goal_state)))
    if heuristic == 2:
        
         sorted_possible_moves,sorted_possible_paths = zip(*sorted(zip(possible_moves,possible_paths),key=lambda x: h2(x[0], goal_state)))

    return sorted_possible_moves,sorted_possible_paths

In [33]:
def solve(curr_state, goal_state, heuristic=0):
   
    visited = np.array([])
    frontier = np.array([curr_state.reshape(-1)])
    dict={str(curr_state.tolist()):"Start->"}
    while (curr_state.tolist() != goal_state.tolist()):
        print(curr_state, end="\n\n")
        possible_moves,possible_paths = get_possible_moves(curr_state,dict[str(curr_state.tolist())])
        if heuristic != 0:
            possible_moves,possible_paths = sort_by_heuristic(possible_moves, possible_paths,goal_state, heuristic)
        for ind, move in enumerate(possible_moves):
            curr_move = move.reshape(-1).tolist()
            if curr_move not in frontier.tolist():
                if curr_move not in visited.tolist():
                    frontier = np.vstack((frontier, move.reshape(-1)))
                    move_taken= possible_paths[ind]
                    dict[str(move.tolist())]=move_taken
        if (frontier == np.array([])):
            print("STRUCK!!!")
            returnd
        if (visited == np.array([])).all():
            visited = np.array(curr_state.reshape(-1,1))
        else:
            visited = np.vstack((visited, curr_state.reshape(-1)))
        frontier = np.delete(frontier, 0, 0)
        curr_state = frontier[0].reshape(3,3)
    print("GOAL REACHED!!!")
    print(curr_state)
    print(dict[str(curr_state.tolist())]+"Goal")

In [34]:
CURR_STATE = generate_instance(GOAL_STATE, 7)
print(CURR_STATE)
solve(CURR_STATE, GOAL_STATE, 1)

[[8 1 2]
 [7 4 3]
 [6 0 5]]
[[8 1 2]
 [7 4 3]
 [6 0 5]]

[[8 1 2]
 [7 0 3]
 [6 4 5]]

[[8 1 2]
 [7 4 3]
 [0 6 5]]

[[8 1 2]
 [7 4 3]
 [6 5 0]]

[[8 1 2]
 [7 4 3]
 [6 0 5]]

[[8 0 2]
 [7 1 3]
 [6 4 5]]

[[8 1 2]
 [0 7 3]
 [6 4 5]]

[[8 1 2]
 [7 3 0]
 [6 4 5]]

[[8 1 2]
 [0 4 3]
 [7 6 5]]

[[8 1 2]
 [7 4 0]
 [6 5 3]]

[[8 1 2]
 [7 0 3]
 [6 4 5]]

[[8 1 2]
 [7 4 3]
 [0 6 5]]

[[8 1 2]
 [7 4 3]
 [6 5 0]]

[[8 2 0]
 [7 1 3]
 [6 4 5]]

[[0 8 2]
 [7 1 3]
 [6 4 5]]

[[0 1 2]
 [8 7 3]
 [6 4 5]]

[[8 1 2]
 [6 7 3]
 [0 4 5]]

[[8 1 0]
 [7 3 2]
 [6 4 5]]

[[8 1 2]
 [7 3 5]
 [6 4 0]]

[[0 1 2]
 [8 4 3]
 [7 6 5]]

[[8 1 2]
 [4 0 3]
 [7 6 5]]

[[8 1 2]
 [7 0 4]
 [6 5 3]]

[[8 1 0]
 [7 4 2]
 [6 5 3]]

[[8 1 2]
 [7 4 3]
 [6 0 5]]

[[8 0 2]
 [7 1 3]
 [6 4 5]]

[[8 1 2]
 [0 7 3]
 [6 4 5]]

[[8 1 2]
 [7 3 0]
 [6 4 5]]

[[8 1 2]
 [0 4 3]
 [7 6 5]]

[[8 1 2]
 [7 4 0]
 [6 5 3]]

[[8 2 3]
 [7 1 0]
 [6 4 5]]

[[7 8 2]
 [0 1 3]
 [6 4 5]]

[[1 0 2]
 [8 7 3]
 [6 4 5]]

[[8 1 2]
 [6 7 3]
 [4 0 5]]

[[8 0 1]
 [7 3 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


[[0 2 3]
 [8 7 1]
 [6 4 5]]

[[8 2 3]
 [6 7 1]
 [0 4 5]]

[[8 2 3]
 [7 0 5]
 [6 1 4]]

[[8 2 3]
 [7 1 5]
 [0 6 4]]

[[7 2 0]
 [1 8 3]
 [6 4 5]]

[[0 7 2]
 [1 8 3]
 [6 4 5]]

[[7 8 2]
 [1 4 3]
 [0 6 5]]

[[7 8 2]
 [1 4 3]
 [6 5 0]]

[[7 8 0]
 [1 3 2]
 [6 4 5]]

[[7 8 2]
 [1 3 5]
 [6 4 0]]

[[7 8 2]
 [6 0 3]
 [4 1 5]]

[[7 8 2]
 [6 1 3]
 [4 5 0]]

[[1 7 2]
 [8 4 3]
 [0 6 5]]

[[1 7 2]
 [8 4 3]
 [6 5 0]]

[[1 7 2]
 [6 8 3]
 [0 4 5]]

[[1 7 0]
 [8 3 2]
 [6 4 5]]

[[1 7 2]
 [8 3 5]
 [6 4 0]]

[[1 2 3]
 [8 0 7]
 [6 4 5]]

[[1 2 3]
 [8 7 5]
 [6 4 0]]

[[0 1 2]
 [8 6 3]
 [4 7 5]]

[[8 2 0]
 [6 1 3]
 [4 7 5]]

[[0 8 2]
 [6 1 3]
 [4 7 5]]

[[8 1 0]
 [6 3 2]
 [4 7 5]]

[[8 1 2]
 [6 3 5]
 [4 7 0]]

[[8 1 2]
 [6 0 7]
 [4 5 3]]

[[8 1 0]
 [6 7 2]
 [4 5 3]]

[[8 3 1]
 [7 4 2]
 [0 6 5]]

[[8 3 1]
 [7 4 2]
 [6 5 0]]

[[8 3 1]
 [7 2 5]
 [6 4 0]]

[[0 3 1]
 [8 7 2]
 [6 4 5]]

[[8 3 1]
 [6 7 2]
 [0 4 5]]

[[7 8 1]
 [3 0 2]
 [6 4 5]]

[[7 8 1]
 [6 3 2]
 [0 4 5]]

[[8 2 0]
 [7 1 5]
 [6 3 4]]

[[0 8 2]
 [7 1

Comparision table of time and memory.
Time taken by the program to find solution usind depth as 2,4,8,16,32 and 64.

In [35]:
print(f"pagesize: {resource.getpagesize()}")
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

pagesize: 4096


1185636

In [ ]:
depths = np.array([2,4,8])
print(depths)
for d in depths:
  start = time.perf_counter()
  CURR_STATE = generate_instance(GOAL_STATE, d)
  solve2(CURR_STATE, GOAL_STATE, 1)
  end = time.perf_counter()
  print(f"Depth: {d}\tTime Taken: {end-start}\tMax Mem: {resource.getrusage(resource.RUSAGE_SELF).ru_maxrss}")


[R, L, U] -> Parent
D -> [R, L, U, D]


16 depth
